# Sketch and Precondition

The sketch-and-precondition approach provides a powerful alternative to both direct factorization methods and the [sketch-and-solve](sketch-and-solve.ipynb) paradigm for solving the linear regression problem {eq}`eqn-regression`.
Unlike sketch-and-solve, which may sacrifice accuracy for speed, sketch-and-precondition maintains high accuracy while achieving significant computational speedups.

The key insight behind sketch-and-precondition is to use a randomized sketch to construct a preconditioner that improves the conditioning the regression problem {eq}`eqn-regression`.

## Iterative Methods and Preconditioning

For any invertible matrix $\vec{M}\in\R^{d\times d}$, the solution to the regression problem {eq}`eqn-regression` is also the solution to the preconditioned least squares problem
```{math}
:label: eqn-precond-leastsquares
\min_{\vec{x}\in\R^d} \|\vec{b}-\vec{A}\vec{M}\vec{y}\|
,\quad \vec{x} = \vec{M}\vec{y}.
```
The advantage of this reformulation is that the condition number of the matrix $\vec{A}\vec{M}$ can be much smaller than that of $\vec{A}$, leading to faster convergence when using iterative methods.

:::{prf:theorem}
LSQR produces an iterate $\vec{x}_t$ such that 
\begin{equation*}
\| \vec{A}(\vec{x}^* - \vec{x}_t) \| \leq \varepsilon \|\vec{b} - \vec{A}\vec{x}^*\|,
\end{equation*}
for some 
\begin{equation*}
t = O\left( \cond(\vec{A}\vec{M}) \log\left(\frac{1}{\varepsilon}\right) \right) ~\text{iterations}.
\end{equation*}
This requires $t$ matrix-vector products with $\vec{A}$, $\vec{A}^\T$, $\vec{M}$, and $\vec{M}^\T$, and an additional $O(tn)$ arithmetic work.
:::


## The Sketch-and-Precondition Algorithm

The basic sketch-and-precondition algorithm follows a two-stage approach: first construct a preconditioner using a randomized sketch, then solve the preconditioned system using an iterative method.

:::{prf:algorithm} Sketch and Precondition
:label: sketch-and-precondition

**Input:** $\vec{A}\in\R^{n\times d}$, $\vec{b}\in\R^n$, sketching dimension $k$, tolerance $\varepsilon$

1. Get $\vec{Q}_1,\vec{R}_1 = \Call{Sketched-QR}(\vec{A},k)$
2. Form preconditioner $\vec{M} = \vec{R}^{-1}$
3. Set $\widehat{\vec{x}} = \Call{Iterative-Method}(\vec{A},\vec{b},\vec{M})$

**Output:** $\vec{x}$
:::

Any version of [Sketched-QR algorithm](../QR-Factorization/randomized-cholesky-qr.ipynb) that produces subspace embedding can be used.

:::{prf:theorem}

Suppose $\vec{A}$ is full-rank and the sketch $\vec{S}$ used by the [Sketched-QR algorithm](../QR-Factorization/randomized-cholesky-qr.ipynb) is an $\varepsilon$-subspace embedding for $\vec{A}$.
Then 
\begin{equation*}
\cond(\vec{A}\vec{M}) \leq  \frac{1+\varepsilon}{1-\varepsilon}.
\end{equation*}
:::

:::{admonition} Proof
:class: dropdown

Recall $\vec{A} = \vec{Q}\vec{R}$.
By {eqn}`sketched-qr-well-conditioned`, $\cond(\vec{A}\vec{M}) = \cond(\vec{A}\vec{R}^{-1}) = \cond(\vec{Q}) \leq (1+\varepsilon)/(1-\varepsilon)$.
:::